In [1]:
import torch
from train import train
from lofi_model import LofiModel
from dataset import MidiDataset
import pretty_midi
import numpy as np
from config import *
print(torch.__version__)
print(torch.version.cuda)


c:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\venv\Lib\site-packages\pretty_midi\instrument.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


2.7.1+cu118
11.8


In [2]:
# Sanity check
dataset = MidiDataset(dataset_dir=r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset")
d1, d2, d3 = dataset[1][0],dataset[1][1], dataset[1][2]
print(d1.shape, d2, d3)


torch.Size([4, 88, 1754]) 1754 71


# TRAINING

In [3]:
if __name__ == "__main__":
    # Device setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

    # Dataset setup
    dataset_dir = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset"

    # Model setup
    model = LofiModel(device)
    model.to(device)

    # Train
    train(model, 
          dataset_dir, 
          verbose=True, 
          model_save_path = "./saved_models/multitrack LSTM-VAE.pth")


Using cuda device
Setting up datasets and dataloaders...
Finished setting up datasets and dataloaders.
Starting training:
The datset has 2517 batches
Epoch [1/500]
Batch index: 1/2517
Current training Loss: 23025.2324
Batch index: 101/2517
Current training Loss: 174926.6781


KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = LofiModel(device)
model.to(device)
model.load_state_dict(torch.load("saved_models/LSTM-VAE (velocity)#1.pth"))
model.to(device)
model.eval()

## Reconstruction

In [ ]:
dataset_dir = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset"
dataset = MidiDataset(dataset_dir)

In [ ]:
original_sequence_tensor, original_length_val, file_path = dataset[11]
file_path

In [ ]:
reconstructions = model.reconstruct(original_sequence_tensor)
reconstructions

In [ ]:
dataset_dir = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset"
dataset = MidiDataset(dataset_dir)

In [ ]:
original_sequence_tensor, original_length_val, file_path = dataset[10]
original_sequence_tensor.shape

In [ ]:
padded_original_sequence, original_length_tensor = MidiDataset.collate_fn([(original_sequence_tensor, original_length_val)])
padded_original_sequence = padded_original_sequence.to(device)
padded_original_sequence.shape

In [ ]:
reconstructed_sample, _, _ = model(padded_original_sequence, original_length_tensor)
reconstructed_sample = reconstructed_sample.squeeze()
reconstructed_sample = torch.sigmoid(reconstructed_sample)
reconstructed_sample[reconstructed_sample < 0.5] = 0
reconstructed_sample[reconstructed_sample >= 0.5] = 1
reconstructed_sample.shape

In [ ]:
reconstructed_sample_T = reconstructed_sample.T
reconstructed_sample_T.shape
reconstructed_sample_T

In [ ]:
file_path

In [ ]:
# Create a PrettyMIDI object
midi = pretty_midi.PrettyMIDI()
piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
piano = pretty_midi.Instrument(program=piano_program)

# Track note on/off times per pitch
# We'll detect note start and end by scanning through time steps
for pitch_idx in range(reconstructed_sample_T.shape[0]):
    note_on = None
    for t in range(reconstructed_sample_T.shape[1]):
        if reconstructed_sample_T[pitch_idx, t] == 1 and note_on is None:
            # Note on at time t/fs seconds
            note_on = t / FS
        elif (reconstructed_sample_T[pitch_idx, t] == 0 or t == reconstructed_sample_T.shape[0]-1) and note_on is not None:
            # Note off at time t/fs seconds
            note_off = t / FS
            # Add the note to the instrument
            note = pretty_midi.Note(    
                velocity=50,
                pitch= pitch_idx+MIN_MIDI_NOTE,
                start=note_on,
                end=note_off
            )
            piano.notes.append(note)
            note_on = None  # reset for next note

    # If a note is still on at the end, close it
    if note_on is not None:
        note_off = reconstructed_sample_T.shape[1] / FS
        note = pretty_midi.Note(
            velocity=50,
            pitch= pitch_idx+MIN_MIDI_NOTE,
            start=note_on,
            end=note_off
        )
        piano.notes.append(note)

# Add instrument to the PrettyMIDI object
midi.instruments.append(piano)

# Write out the MIDI data
midi.write("1 4 6 5 - chord progression-RECONSTRUCTION.mid")

# CHORD EXTRACTOR DEMO

In [ ]:
from music21 import converter

midi_pth = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset\1 2 6 - chord progression.mid"
midi_file = converter.parse(midi_pth)

chords = midi_file.chordify()

chord_progression = []

for c in chords.flat.getElementsByClass("Chord"):
    if not c.isRest:
        root_note = c.root().name
        quality = c.quality
        full_chord_name = c.pitchedCommonName
        offset = c.offset  

        chord_progression.append([root_note, quality, full_chord_name, offset])

chord_progression


In [ ]:
import numpy as np
array = np.load("../midi/98f90636c139256f4b7dade28ab87088.npz")
print(array)
for a in array.keys():
    print(a)
    print(array[a])
    print(20*"___")

In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt

def load_all_tracks_from_npz(npz_path):
    """
    Load all sparse pianoroll tracks from an LPD .npz file into dense arrays.

    Returns:
        dict[int, np.ndarray]: Mapping from track_id to dense pianoroll (shape: time x pitch)
    """
    data = np.load(npz_path)
    tracks = {}

    for key in data.files:
        if key.endswith("_csc_shape"):
            track_id = int(key.split("_")[1])
            shape = data[f"pianoroll_{track_id}_csc_shape"]
            indptr = data[f"pianoroll_{track_id}_csc_indptr"]
            indices = data[f"pianoroll_{track_id}_csc_indices"]

            if f"pianoroll_{track_id}_csc_data" in data:
                values = data[f"pianoroll_{track_id}_csc_data"]
            else:
                values = np.ones_like(indices, dtype=np.uint8)

            sparse = csc_matrix((values, indices, indptr), shape=shape)
            dense = sparse.toarray()
            tracks[track_id] = dense

    return tracks

def plot_all_tracks(tracks_dict):
    """
    Plot all tracks in the same figure, stacked vertically.
    Each track is shown as a separate pianoroll.
    """
    num_tracks = len(tracks_dict)
    fig, axes = plt.subplots(num_tracks, 1, figsize=(12, 2.5 * num_tracks), sharex=True)

    if num_tracks == 1:
        axes = [axes]

    for i, (track_id, pianoroll) in enumerate(sorted(tracks_dict.items())):
        ax = axes[i]
        ax.imshow(pianoroll.T, aspect='auto', origin='lower', cmap='Greys')
        ax.set_title(f"Track {track_id}")
        ax.set_ylabel("Pitch")
        ax.set_xlabel("Time step")

    plt.tight_layout()
    plt.show()



In [ ]:
npz_path = "../midi/ea04f8ebe8f52d78ab6ce59d9ba04d20.npz"

tracks = load_all_tracks_from_npz(npz_path)
plot_all_tracks(tracks)


In [ ]:
npz_path = "../midi/98f90636c139256f4b7dade28ab87088.npz"

tracks = load_all_tracks_from_npz(npz_path)
plot_all_tracks(tracks)